# ICON Stencil
This stencil is based on `model/atmosphere/dycore/src/icon4py/model/atmosphere/dycore/mo_solve_nonhydro_stencil_20.py`.
It is very simple and most faithfull implementation.

A reference `NumPy` implementation can be found in `model/atmosphere/dycore/tests/stencil_tests/test_mo_velocity_advection_stencil_20.py`.


In [1]:
import os
ncpu=1
os.environ['XLA_FLAGS'] = f"--xla_force_host_platform_device_count={ncpu}"
os.environ["XLA_FLAGS"] = ("--xla_cpu_multi_thread_eigen=false "
                           "intra_op_parallelism_threads=1")
os.environ['JAX_PLATFORMS'] = "cpu"

import numpy as np
import jax
import sys
import jax.numpy as jnp
from jax import jit, grad, vmap
from jax import random

# This must be enabled when `make_jaxpr` is called, because otherwhise we get problems.
jax.config.update("jax_enable_x64", True)

import dace


In [2]:
from JaxprToSDFG import  JaxprToSDFG
t = JaxprToSDFG()

### Demo Input

### Grid Dimensions
Since we miss the GT4Py Dimension mechanism, we will make some tricks.
- `{K, Cell, Edge, Vertex}DimN` is the number of either Vertex, Cell, Edges or vertical levels.
- Valid indexes always life within the range `[0, N-1]` where `N` is the above entity.
- However, they do not denote at which position they are.



In [3]:
from icondims import SimpleGridData

### Offset Providers
Here we define the offset providors, i.e. the connectivity.
It is important that they have to be read in a certain way.

In [4]:
SimpleGridData.c2v_table.shape

(18, 3)

In [5]:


# As stated above, each edge is either adijacent to 1 cell (at the boundary) or 2 (in the inner part)
#  This table describes which cell are adijacent to a certain edge, in case it is undefined we have `-1`.  
edge_to_cell_table = np.array(SimpleGridData.e2c_table).astype(np.int32)

# We have a triangular grid, this means a cell is bounded by three edges.
#  In unstructured it could be, we have many geometrical forms, in this case
#  we would allocate the matrix such that it fits the largest one and fill up with `-1` if we do not need them.
cell_to_edge_table = np.array(SimpleGridData.c2e_table).astype(np.int32)




In [6]:
KDimN = 50     # Vertical levels
CellDimN = cell_to_edge_table.shape[0]   # How many cells we have
EdgeDimN = edge_to_cell_table.shape[0]
VertexDimN = SimpleGridData.v2c_table.shape[0]


# `E2C` is the offset map that transforms _edges_ to _cells_.
#   Since a edge is adijacent to at most two cells this dimension is 2 but at least 1.
E2CDimN = 2


In [7]:
# This is the level mask, the `+ 1` is taken from the test but it makes sense, at least a bit.
#  For `levelmask[Koff]` use `levelmask[:-1]`
#  and for `levelmask[Koff[+1]]` use `levelmask[1:]`.
levelmask = (np.random.rand(KDimN + 1) < 0.5).astype(np.bool_)

### Fields
These are the fields we are using as an inoput material

In [8]:
# Field defined on the edges of the grid and different in every hight level.
w_con_e = np.random.rand(EdgeDimN, KDimN).astype(np.float64)

# This is a field, which is the same for every height level.
#  It is defined for every edge but for each edge it has two values (thus `E2CDimN`) which is the result
#  of an cell to edge mapping.
c_lin_e = np.random.rand(EdgeDimN, E2CDimN).astype(np.float64)

# This is a field that defines for every cell in every height level one value.
z_w_con_c_full = np.random.rand(CellDimN, KDimN).astype(np.float64)

ddqz_z_full_e = np.random.rand(EdgeDimN, KDimN).astype(np.float64)

cfl_w_limit = np.full((1), 4.0)
scalfac_exdiff = np.full((1), 6.0)
dtime = np.full((1), 2.0)


# Implementation

### Python Implementation

In [9]:
# Note that this is only the first part of the stencil.
def TestStencil(c_lin_e: dace.float64[EdgeDimN, E2CDimN],
                w_con_e: dace.float64[EdgeDimN, KDimN],
                z_w_con_c_full: dace.float64[CellDimN, KDimN],
                levelmask: dace.bool_[KDimN+1],
                edge_to_cell_table: dace.int32[EdgeDimN, E2CDimN],
                
                ddqz_z_full_e: dace.float64,    # Ver 2
                cfl_w_limit: dace.float64,
                scalfac_exdiff: dace.float64,
                dtime: dace.float64,
):
    c_lin_e = jnp.expand_dims(c_lin_e, axis=-1)   # Adding a new dimensions of shape 1 at the end for propper broadcasting (taken from the test).
    #difcoef = jnp.zeros_like(w_con_e)
    
    #levelmask_offset_0 = levelmask[:-1]   # This is `levelmask[Koff[0]]`, i.e. the normal field
    #levelmask_offset_1 = levelmask[1:]    # This is `levelmask[Koff[1]]`, i.e. the field at the top.

    if(False):
        # This is more general but it requieres that a `scatter` operation is present.
        (EdgeDimN, E2CDimN, _) = c_lin_e.shape
        (CellDimN, KDimN)      = z_w_con_c_full.shape
        z_w_con_c_full_b = jnp.zeros((EdgeDimN, E2CDimN, KDimN))
        z_w_con_c_full_b = z_w_con_c_full_b.at[:, 0, :].set(z_w_con_c_full[edge_to_cell_table[:, 0], :]) 
        z_w_con_c_full_b = z_w_con_c_full_b.at[:, 1, :].set(z_w_con_c_full[edge_to_cell_table[:, 1], :])  # The `-1` in the `edge_to_cell_table` will be resolved as some bullshit values, that will be removed further down in the where.
        
    elif(True):
        z_w_0_ = z_w_con_c_full[edge_to_cell_table[:, 0], :]
        z_w_1_ = z_w_con_c_full[edge_to_cell_table[:, 1], :]
        z_w_con_c_full_0 = jnp.expand_dims(z_w_0_, axis=1)
        z_w_con_c_full_1 = jnp.expand_dims(z_w_1_, axis=1)
        z_w_con_c_full_b = jnp.concatenate((z_w_con_c_full_0, z_w_con_c_full_1), axis=1)
    #

    Z = jnp.where(
                jnp.expand_dims(edge_to_cell_table != -1, axis=-1), #(grid.connectivities[E2CDim] != -1)[:, :, np.newaxis],
                c_lin_e * z_w_con_c_full_b,    #z_w_con_c_full[grid.connectivities[E2CDim]],
                0,
    )
    if(True):
        # This is knoledge that we sum over the non iterated direction.
        Y = Z[:, 0, :] + Z[:, 1, :]
    else:
        Y = jnp.sum(Z, axis=1)

    w_con_e = jnp.where((levelmask[:-1]) | (levelmask[1:]), Y, w_con_e)
    difcoef = jnp.where(
        (levelmask[:-1]) | (levelmask[1:]) & (jnp.abs(w_con_e) > cfl_w_limit * ddqz_z_full_e),
        scalfac_exdiff
        * jnp.minimum(
            0.85 - cfl_w_limit * dtime,
            jnp.abs(w_con_e) * dtime / ddqz_z_full_e - cfl_w_limit * dtime,
        ),
        0,
    )
    return difcoef
#

In [10]:
with jax.disable_jit(disable=True):
    TestStencil_jaxpr = jax.make_jaxpr(TestStencil)(c_lin_e, w_con_e,
                                                    z_w_con_c_full,
                                                    levelmask, edge_to_cell_table, ddqz_z_full_e, cfl_w_limit[0], scalfac_exdiff[0], dtime[0])
print(TestStencil_jaxpr)

{ lambda ; a:f64[27,2] b:f64[27,50] c:f64[18,50] d:bool[51] e:i32[27,2] f:f64[27,50]
    g:f64[] h:f64[] i:f64[]. let
    j:f64[27,2,1] = broadcast_in_dim[
      broadcast_dimensions=(0, 1)
      shape=(27, 2, 1)
    ] a
    k:i32[27,1] = slice[limit_indices=(27, 1) start_indices=(0, 0) strides=None] e
    l:i32[27] = squeeze[dimensions=(1,)] k
    m:bool[27] = lt l 0
    n:i32[27] = add l 18
    o:i32[27] = select_n m l n
    p:i32[27,1] = broadcast_in_dim[broadcast_dimensions=(0,) shape=(27, 1)] o
    q:f64[27,50] = gather[
      dimension_numbers=GatherDimensionNumbers(offset_dims=(1,), collapsed_slice_dims=(0,), start_index_map=(0,))
      fill_value=None
      indices_are_sorted=False
      mode=GatherScatterMode.PROMISE_IN_BOUNDS
      slice_sizes=(1, 50)
      unique_indices=False
    ] c p
    r:i32[27,1] = slice[limit_indices=(27, 2) start_indices=(0, 1) strides=None] e
    s:i32[27] = squeeze[dimensions=(1,)] r
    t:bool[27] = lt s 0
    u:i32[27] = add s 18
    v:i32[27] = 

In [11]:
TestStencil_sdfg = t(TestStencil_jaxpr, simplify=True, auto_opt=5)
TestStencil_sdfg_unopt = t(TestStencil_jaxpr)

In [12]:
resExp = TestStencil(c_lin_e, w_con_e, z_w_con_c_full, levelmask, edge_to_cell_table, ddqz_z_full_e, cfl_w_limit[0], scalfac_exdiff[0], dtime[0])
resDC  = TestStencil_sdfg(c_lin_e, w_con_e, z_w_con_c_full, levelmask, edge_to_cell_table, ddqz_z_full_e, cfl_w_limit[0], scalfac_exdiff[0], dtime[0])

assert np.allclose(resExp, resDC)


In [13]:
if(True):
    TestStencil_sdfg.view()
else:
    TestStencil_sdfg.save("./SDFG_of_stencil.json")


File saved at /tmp/tmpi4rybcm9.sdfg.html


### Dace

In [14]:
def TestStencil_NP(c_lin_e: dace.float64[EdgeDimN, E2CDimN],
                w_con_e: dace.float64[EdgeDimN, KDimN],
                z_w_con_c_full: dace.float64[CellDimN, KDimN],
                levelmask: dace.bool_[KDimN+1],
                edge_to_cell_table: dace.int32[EdgeDimN, E2CDimN],
                
                ddqz_z_full_e: dace.float64,    # Ver 2
                cfl_w_limit: dace.float64,
                scalfac_exdiff: dace.float64,
                dtime: dace.float64,
):

    levelmask_offset_0 = levelmask[:-1]
    levelmask_offset_1 = levelmask[1:]

    c_lin_e = np.expand_dims(c_lin_e, axis=-1)

    w_con_e = np.where(
        (levelmask_offset_0) | (levelmask_offset_1),
        np.sum(
            np.where(
                jnp.expand_dims(edge_to_cell_table != -1, axis=-1), #(grid.connectivities[E2CDim] != -1)[:, :, np.newaxis],
                c_lin_e * z_w_con_c_full[grid.connectivities[E2CDim]],
                0,
            ),
            axis=1,
        ),
        w_con_e,
    )
    difcoef = np.where(
        ((levelmask_offset_0) | (levelmask_offset_1))
        & (np.abs(w_con_e) > cfl_w_limit * ddqz_z_full_e),
        scalfac_exdiff
        * np.minimum(
            0.85 - cfl_w_limit * dtime,
            np.abs(w_con_e) * dtime / ddqz_z_full_e - cfl_w_limit * dtime,
        ),
        difcoef,
    )

In [15]:
#resExp = TestStencil_NP(c_lin_e, w_con_e, z_w_con_c_full, levelmask, edge_to_cell_table, ddqz_z_full_e, cfl_w_limit[0], scalfac_exdiff[0], dtime[0])

### GT4Py Implementation

In [16]:
import gt4py
from gt4py.next.common import GridType
from gt4py.next.ffront.decorator import field_operator, program
from gt4py.next.ffront.fbuiltins import (  # noqa: A004 # import gt4py builtin
    Field,
    abs,
    broadcast,
    int32,
    minimum,
    neighbor_sum,
    where,
)

from icondims import (
    E2C,
    E2C2EO,
    E2V,
    CellDim,
    E2C2EODim,
    E2CDim,
    EdgeDim,
    KDim,
    Koff,
    VertexDim,
)

E2C_offset_provider = gt4py.next.NeighborTableOffsetProvider(edge_to_cell_table, EdgeDim, CellDim, 2)


@field_operator
def _mo_velocity_advection_stencil_20(
    levelmask: Field[[KDim], bool],
    c_lin_e: Field[[EdgeDim, E2CDim], float],
    z_w_con_c_full: Field[[CellDim, KDim], float],
    
    ddqz_z_full_e: Field[[EdgeDim, KDim], float],
    cfl_w_limit: float,
    scalfac_exdiff: float,
    dtime: float,
) -> Field[[EdgeDim, KDim], float]:
    w_con_e = broadcast(0.0, (EdgeDim, KDim))
    difcoef = broadcast(0.0, (EdgeDim, KDim))
    w_con_e = where(
        levelmask | levelmask(Koff[1]),
        neighbor_sum(c_lin_e * z_w_con_c_full(E2C), axis=E2CDim),
        w_con_e,
    )

    difcoef = where(
        (levelmask | levelmask(Koff[1])) & (abs(w_con_e) > cfl_w_limit * ddqz_z_full_e),
        scalfac_exdiff
        * minimum(
            0.85 - cfl_w_limit * dtime,
            abs(w_con_e) * dtime / ddqz_z_full_e - cfl_w_limit * dtime,
        ),
        difcoef,
    )
    
    return difcoef
#


@program(grid_type=GridType.UNSTRUCTURED)
def mo_velocity_advection_stencil_20(
    levelmask: Field[[KDim], bool],
    c_lin_e: Field[[EdgeDim, E2CDim], float],
    z_w_con_c_full: Field[[CellDim, KDim], float],

    ddqz_z_full_e: Field[[EdgeDim, KDim], float],
    cfl_w_limit: float,
    scalfac_exdiff: float,
    dtime: float,
    
    _ret_: Field[[EdgeDim, KDim], float],
    horizontal_start: int32,
    horizontal_end: int32,
    vertical_start: int32,
    vertical_end: int32,
):
    _mo_velocity_advection_stencil_20(
        levelmask,
        c_lin_e,
        z_w_con_c_full,
        ddqz_z_full_e, cfl_w_limit, scalfac_exdiff, dtime,
        out=_ret_,
        domain={
            EdgeDim: (horizontal_start, horizontal_end),
            KDim: (vertical_start, vertical_end),
        },
    )

In [17]:
from gt4py.next.program_processors.runners.dace_iterator import run_dace_iterator

levelmask_F = gt4py.next.as_field([KDim], levelmask)
c_lin_e_F = gt4py.next.as_field([EdgeDim, E2CDim], c_lin_e)
z_w_con_c_full_F = gt4py.next.as_field([CellDim, KDim], z_w_con_c_full)

ddqz_z_full_e_F = gt4py.next.as_field([EdgeDim, KDim], ddqz_z_full_e)

_ret_ = gt4py.next.as_field([EdgeDim, E2CDim], np.zeros_like(resExp))


In [18]:
TestStencil_itir_sdfg = run_dace_iterator(
    mo_velocity_advection_stencil_20.itir,
    levelmask_F, c_lin_e_F, z_w_con_c_full_F,

    ddqz_z_full_e_F, cfl_w_limit[0], scalfac_exdiff[0], dtime[0],

    _ret_,
    0, EdgeDimN, 0, KDimN,

    return_sdfg=True, run_sdfg=True,
    offset_provider={'E2C':E2C_offset_provider, "Koff": KDim})

Will return the generated SDFG.
